In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.layers import BatchNormalization
from keras.wrappers.scikit_learn import KerasClassifier
import tensorflow as tf
import cv2
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import serial
import time

In [22]:
def detect_laser_points():
    # Open the webcam
    cap = cv2.VideoCapture(0)

    # Check if the webcam is opened successfully
    if not cap.isOpened():
        print("Failed to open the webcam")
        return

    while True:
        # Read a frame from the webcam
        ret, frame = cap.read()

        # If the frame was not captured successfully, break the loop
        if not ret:
            print("Failed to capture frame from webcam")
            break

        # Convert the frame to Gray scale image
        gry = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # defining the range or threshold
        mask = cv2.inRange(gry, 225,255,cv2.THRESH_BINARY)
        # Perform morphological operations to remove noise
        mask = cv2.erode(mask, None, iterations=2)
        mask = cv2.dilate(mask, None, iterations=2)

        # Find contours in the masked image
        contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Process each contour
        for contour in contours:
            # Calculate the area of the contour
            area = cv2.contourArea(contour)

            # Filter out small contours
            if area > 10000:
                # Find the centroid of the contour
                M = cv2.moments(contour)
                centroid_x = int(M["m10"] / M["m00"])
                centroid_y = int(M["m01"] / M["m00"])
                data = f"{centroid_x},{centroid_y}\n"
                # Draw a circle at the centroid
                cv2.circle(frame, (centroid_x, centroid_y), round((area/(np.pi))**0.5), (255, 255, 255), 2)
                cv2.putText(frame,(str(centroid_x)+','+str(centroid_y)),(centroid_x+20,centroid_y+30),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
                arduino.write(data.encode())
        # Display the frame with laser points
        cv2.imshow("Laser Point Detection", frame)
        time.sleep(0.005)
        # Check for the 'q' key press to exit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close any open windows
    cap.release()
    cv2.destroyAllWindows()

# Call the red laser point detection function
arduino = serial.Serial('COM4', 9600)

arduino.timeout=0.005
detect_laser_points()
arduino.close()